In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

# Define the repository URL and the local directory where you want to clone/pull the repository
repo_url = 'https://github.com/aakashvardhan/s11-gradcam.git'
local_dir = '/content/s11-gradcam'



# Check if the local directory already exists
if not os.path.exists(local_dir):
    # Clone the repository because it does not exist
    !git clone {repo_url}
else:
    # Change directory to the local repository
    %cd {local_dir}
    # Pull the latest changes because the repository already exists
    !git pull

In [ ]:
import sys
sys.path.append('/content/s11-gradcam')
sys.path.append('/content/s11-gradcam/utils')

In [ ]:
from utils.misclassification import show_misclassified_images, plt_misclassified_images
from utils.gradcam import display_gradcam_output
from config import get_config
config = get_config()

In [ ]:
misclass_imgs, misclass_targets, misclass_preds = show_misclassified_images(model, test_loader, config)
plt_misclassified_images(config, misclass_imgs, misclass_targets, misclass_preds)